In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import sqlalchemy
pd.set_option('display.max_rows',500)
print("✅ All imports working. Environment is set up correctly.")


✅ All imports working. Environment is set up correctly.


In [2]:
from dotenv import load_dotenv
import os

# Load variables from the .env file in your project root
load_dotenv()

# Retrieve each variable
BASE_URL = os.getenv("EIRGRID_BASE_URL")
DB_URL = os.getenv("DATABASE_URL")
LOCAL_TZ = os.getenv("LOCAL_TZ")

print("Base URL:", BASE_URL)
print("Database URL:", DB_URL)
print("Local timezone:", LOCAL_TZ)


Base URL: https://api.eirgrid.ie/smartgrid/data
Database URL: sqlite:///db/eirgrid.db
Local timezone: Europe/Dublin


### API Data Contract (First Draft)

- **Base URL:**  
  `https://www.smartgriddashboard.com/api/chart/`

- **Endpoint Path:**  
  `/chart/` (single endpoint used with query parameters)

- **Required Parameters (with examples):**
  | Parameter | Example | Description |
  |------------|----------|-------------|
  | `region` | `ALL` | Geographic region for data (use `ALL` for system-wide totals) |
  | `chartType` | `default` | Chart type; usually left as `default` |
  | `dateRange` | `day` | Range of data to fetch (`day`, `week`, etc.) |
  | `dateFrom` | `24-Oct-2025` | Start date (**DD-MMM-YYYY**) |
  | `dateTo` | `24-Oct-2025` | End date (**DD-MMM-YYYY**) |
  | `areas` | `solaractual,windactual,demandactual` | Comma-separated list of metrics to fetch |

- **Example Request URL:**  



- **Response Structure (JSON):**
```json
{
  "Rows": [
    {
      "Value": 0,
      "Region": "ALL",
      "EffectiveTime": "23-Oct-2025 00:00:00",
      "FieldName": "SOLAR_ACTUAL"
    },
    {
      "Value": 0,
      "Region": "ALL",
      "EffectiveTime": "23-Oct-2025 00:15:00",
      "FieldName": "SOLAR_ACTUAL"
    },
    ...
  ]
}


In [3]:
base_url = 'https://www.smartgriddashboard.com/api/chart/'

params = {'region':'ALL',
 'chartType':'default',
 'dateRange':'day',
 'dateFrom': '23-Oct-2025',
 'dateTo': '23-Oct-2025',
 'areas': 'windactual',
}

print(base_url)

for key in params:
    print(key)
    
print(len(params))
 
    
            

https://www.smartgriddashboard.com/api/chart/
region
chartType
dateRange
dateFrom
dateTo
areas
6


In [4]:
response = requests.get(base_url, params=params, timeout=30)
print("Status code:", response.status_code)

if response.ok:
    data = response.json() 
    print("Top-level keys:", list(data.keys()))
    print("Sample rows:", data["Rows"][0])

Status code: 200
Top-level keys: ['Rows']
Sample rows: {'Value': 653, 'Region': 'ALL', 'EffectiveTime': '23-Oct-2025 00:00:00', 'FieldName': 'WIND_ACTUAL'}


In [5]:
rows_list = data['Rows']
df_raw = pd.DataFrame(rows_list)
print(df_raw.head())
print(df_raw.dtypes)

   Value Region         EffectiveTime    FieldName
0    653    ALL  23-Oct-2025 00:00:00  WIND_ACTUAL
1    753    ALL  23-Oct-2025 00:15:00  WIND_ACTUAL
2    959    ALL  23-Oct-2025 00:30:00  WIND_ACTUAL
3   1101    ALL  23-Oct-2025 00:45:00  WIND_ACTUAL
4   1235    ALL  23-Oct-2025 01:00:00  WIND_ACTUAL
Value             int64
Region           object
EffectiveTime    object
FieldName        object
dtype: object


In [6]:
df_raw = df_raw.rename(columns = {'Value':'value',
                         'Region':'region',
                         'EffectiveTime':'ts_local_str',
                         'FieldName':'metric'})
df_raw['metric'] = df_raw['metric'].str.lower()
df_raw["unit"] = "MW"
df_raw.head()

,value,region,ts_local_str,metric,unit
0,653,ALL,23-Oct-2025 00:00:00,wind_actual,MW
1,753,ALL,23-Oct-2025 00:15:00,wind_actual,MW
2,959,ALL,23-Oct-2025 00:30:00,wind_actual,MW
3,1101,ALL,23-Oct-2025 00:45:00,wind_actual,MW
4,1235,ALL,23-Oct-2025 01:00:00,wind_actual,MW


In [7]:
df_raw['ts_local_str'] = pd.to_datetime(df_raw['ts_local_str'], format = "%d-%b-%Y %H:%M:%S")
print(df_raw.dtypes)

value                    int64
region                  object
ts_local_str    datetime64[ns]
metric                  object
unit                    object
dtype: object


In [8]:
# Alternative method
# df_raw['ts_local_str'] = df_raw['ts_local_str'].apply(lambda x: pd.to_datetime(x, format="%d-%b-%Y %H:%M:%S"))


In [9]:
df_raw['ts_local'] = df_raw['ts_local_str'].dt.tz_localize("Europe/Dublin",  
                                                   nonexistent = "shift_forward",
                                                   ambiguous = "NaT")

df_raw["ts_local"].dtype

datetime64[ns, Europe/Dublin]

In [10]:
df_raw["ts_utc"] = df_raw["ts_local"].dt.tz_convert("UTC")
df_raw["ts_utc"].dtype

datetime64[ns, UTC]

Timezone Policy & DST Handling

All timestamps are localized to Europe/Dublin and stored as UTC for consistency.

During the spring-forward transition, any nonexistent local times are shifted forward to the next valid time (nonexistent="shift_forward").

During the fall-back transition, ambiguous local times are marked as missing (ambiguous="NaT").

These rules ensure database timestamps remain continuous and non-duplicated.

In [11]:
# Check that UTC is an hour ahead of local time
print("order:", df_raw[["ts_local", "ts_utc"]].head())

# Check that UTC is constantly increasing (will return True if it is)
print("monotonicity:", df_raw["ts_utc"].is_monotonic_increasing)

# Check the difference shows a 1-hour offset
print("difference:", (df_raw["ts_local"] - df_raw["ts_utc"]).unique())



order:                    ts_local                    ts_utc
0 2025-10-23 00:00:00+01:00 2025-10-22 23:00:00+00:00
1 2025-10-23 00:15:00+01:00 2025-10-22 23:15:00+00:00
2 2025-10-23 00:30:00+01:00 2025-10-22 23:30:00+00:00
3 2025-10-23 00:45:00+01:00 2025-10-22 23:45:00+00:00
4 2025-10-23 01:00:00+01:00 2025-10-23 00:00:00+00:00
monotonicity: True
difference: <TimedeltaArray>
['0 days']
Length: 1, dtype: timedelta64[ns]


In [12]:
df_raw[["ts_local", "ts_utc"]].head()


,ts_local,ts_utc
0,2025-10-23 00:00:00+01:00,2025-10-22 23:00:00+00:00
1,2025-10-23 00:15:00+01:00,2025-10-22 23:15:00+00:00
2,2025-10-23 00:30:00+01:00,2025-10-22 23:30:00+00:00
3,2025-10-23 00:45:00+01:00,2025-10-22 23:45:00+00:00
4,2025-10-23 01:00:00+01:00,2025-10-23 00:00:00+00:00


In [13]:
columns_selected = ['ts_utc','metric','value','unit','region','ts_local']
df_raw = df_raw.drop("ts_local_str",axis=1)
df_raw = df_raw.sort_values("ts_utc").reset_index(drop=True)
df_tidy = df_raw[columns_selected].copy()

print(df_tidy.head())
print(len(df_tidy))
print(df_tidy["ts_utc"].diff().value_counts().head())
print(df_tidy.duplicated(subset=["ts_utc","metric"]).sum())
print(df_tidy.isna().sum())

                     ts_utc       metric  value unit region  \
0 2025-10-22 23:00:00+00:00  wind_actual    653   MW    ALL   
1 2025-10-22 23:15:00+00:00  wind_actual    753   MW    ALL   
2 2025-10-22 23:30:00+00:00  wind_actual    959   MW    ALL   
3 2025-10-22 23:45:00+00:00  wind_actual   1101   MW    ALL   
4 2025-10-23 00:00:00+00:00  wind_actual   1235   MW    ALL   

                   ts_local  
0 2025-10-23 00:00:00+01:00  
1 2025-10-23 00:15:00+01:00  
2 2025-10-23 00:30:00+01:00  
3 2025-10-23 00:45:00+01:00  
4 2025-10-23 01:00:00+01:00  
96
ts_utc
0 days 00:15:00    95
Name: count, dtype: int64
0
ts_utc      0
metric      0
value       0
unit        0
region      0
ts_local    0
dtype: int64


In [14]:
find_null = df_tidy.isnull()
print(find_null)

    ts_utc  metric  value   unit  region  ts_local
0    False   False  False  False   False     False
1    False   False  False  False   False     False
2    False   False  False  False   False     False
3    False   False  False  False   False     False
4    False   False  False  False   False     False
5    False   False  False  False   False     False
6    False   False  False  False   False     False
7    False   False  False  False   False     False
8    False   False  False  False   False     False
9    False   False  False  False   False     False
10   False   False  False  False   False     False
11   False   False  False  False   False     False
12   False   False  False  False   False     False
13   False   False  False  False   False     False
14   False   False  False  False   False     False
15   False   False  False  False   False     False
16   False   False  False  False   False     False
17   False   False  False  False   False     False
18   False   False  False  Fals

In [15]:
find_null = df_tidy["value"].isnull()
print(find_null)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
60    False
61    False
62    False
63    False
64    False
65    False
66    False
67    False
68    False
69    False
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
79    False
80    False
81    False
82    False
83  

In [16]:
print(df_tidy["value"].dtype)

int64


In [17]:
cols = ["ts_utc", "ts_local", "value"]

print(df_tidy.loc[find_null, cols].head(2))
print(df_tidy.loc[find_null, cols].tail(2))

Empty DataFrame
Columns: [ts_utc, ts_local, value]
Index: []
Empty DataFrame
Columns: [ts_utc, ts_local, value]
Index: []


## Ingestion Policy

**Truth-first (staging):**  
- Capture exactly what the API returns into a *staging* table.  
- Preserve `NULL` values (no imputation).  
- Store timestamps in **UTC** only.  
- Record `source` and `ingested_at`.

**Promotion to facts (complete days only):**  
- For each `(metric, region, calendar_day_local)`, promote to *facts* only when we have **96** readings (15-min cadence × 24 h) and no duplicates.  
- “Today” is typically partial → stays in staging; promoted on the next run.

**Idempotent upsert (facts):**  
- Unique key: `(ts_utc, metric_id, region_id)`.  
- If a duplicate arrives with the same key:  
  - Keep the **newest non-NULL** value (do not overwrite non-NULL with NULL).  
  - Update `ingested_at`.

**DST & timezone:**  
- Localize to `Europe/Dublin`, then convert to **UTC**;  
  `nonexistent="shift_forward"`, `ambiguous="NaT"`.  
- Store UTC in DB; derive local time in views.

**Quality & completeness:**  
- Track per-day completeness (`expected=96`, `observed`, `nulls`, `is_complete`).  
- No imputation in ingestion; any interpolation happens later with an `is_imputed` flag.
